In [ ]:
import os
import sys 
from pystac import Catalog, Collection, Item, MediaType, Asset, CatalogType
import gdal
import numpy as np
import datetime
from helpers import *
import logging
import osr, ogr

from shapely.wkt import loads
from shapely.geometry import box

In [ ]:
logging.basicConfig(stream=sys.stderr, 
                    level=logging.DEBUG,
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    datefmt='%Y-%m-%dT%H:%M:%S')

In [ ]:
gdal.UseExceptions() 

In [ ]:
os.environ['PREFIX']='/opt/anaconda/envs/env_ewf_satcen_03_03_02'

In [ ]:
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')
os.environ['GDAL_DATA'] = os.path.join(os.environ['PREFIX'], 'share/gdal')


In [ ]:
service = dict([('title', 'Sentinel-2 burned area identification'),
                ('abstract', 'This is a short description'),
                ('id', 'ewf-satcen-03-03-02')])

In [ ]:
pp_threshold = dict([('identifier', 'pp_threshold'),
                     ('title', 'Post Processing threshold in pixels'),
                     ('abstract', 'Number of pixels composing the isolated polygon to be removed (if 0 no post processing is applied)'),
                     ('value', '3'),
                     ('min_occurs', '1'),
                     ('max_occurs', '1')])

In [ ]:
ndvi_threshold = dict([('identifier', 'ndvi_threshold'),
                       ('value', '0.19'),
                       ('title', 'NDVI difference threshold'),
                       ('abstract', 'NDVI difference threshold'),
                       ('min_occurs', '1'),
                       ('max_occurs', '1')])

In [ ]:
ndwi_threshold = dict([('identifier', 'ndwi_threshold'),
                       ('value', '0.18'),
                       ('title', 'NDWI difference threshold'),
                       ('abstract', 'NDWI difference threshold'),
                       ('min_occurs', '1'),
                       ('max_occurs', '1')])

In [ ]:
wkt = dict([('identifier', 'aoi'),
            ('value', 'POLYGON((149.74042460751588 -34.29772543048931,150.93246853304504 -34.323665099129535,150.90758708373184 -35.313155442237914,149.70124915286058 -35.28624837182783,149.74042460751588 -34.29772543048931))'),
            ('title', 'Area of interest'),
            ('abstract', 'Area of interest in WKT or bounding box'),
            ('min_occurs', '0'),
            ('max_occurs', '1')])

In [ ]:
pre_event = dict([('identifier', 'pre_event'),
                  ('title', 'Sentinel-2 Level-2A pre-event'),
                  ('abstract', 'Sentinel-2 Level-2A pre-event acquisition'),
                  ('value', 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191101T000241_N0213_R030_T56HKG_20191101T020007'),
                  ('min_occurs', '1'),
                  ('max_occurs', '1'),
                  ('stac:collection', 'pre-event'),
                  ('stac:href', 'catalog.json')])

In [ ]:
post_event = dict([('identifier', 'post_event'),
                   ('title', 'Sentinel-2 Level-2A post-event'),
                   ('abstract', 'Sentinel-2 Level-2A post-event acquisition'),
                   #('value', 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20200320T000241_N0214_R030_T56HKG_20200320T020042'),
                   ('value', 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191231T000241_N0213_R030_T56HKG_20191231T015159'),
                   ('min_occurs', '1'),
                   ('max_occurs', '1'),
                   ('stac:collection', 'post-event'),
                   ('stac:href', 'catalog.json')])

In [ ]:
data_path = '/workspace/data/s2/'

In [ ]:
input_catalog = '/workspace/data/s2/catalog.json'

In [ ]:
logging.info(input_catalog)

cat = Catalog.from_file(input_catalog)

if cat is None:
    raise ValueError()

logging.info(cat.describe())


In [ ]:
collections = []
dates = []
for col in iter(cat.get_children()):

    collections.append(col)

    item = next(col.get_items())

    print(item.assets['SCL'])
    
    if col.id == 'pre-event':
        pre_date = item.datetime.strftime("%Y-%m-%dT%H:%M:%SZ")
        masterID = item.id
    else:
        post_date = item.datetime.strftime("%Y-%m-%dT%H:%M:%SZ")
        slaveID = item.id

    #dates.append(item.datetime)
    #extent=col.extent
    #geometry = item.geometry
    #bbox = item.bbox

if len(collections) == 0:
    raise ValueError()
    

In [ ]:
if wkt['value'] != "":
    
    bbox = loads(wkt['value']).bounds
    logging.info('Burned Area will be cropped over {}'.format(wkt['value']))

In [ ]:
for collection in collections:

    item = next(collection.get_items())
    
    logging.info('Stacking bands for input {}'.format(collection.id))
    
    vrt_bands = []

    
    
    for band in ['B04', 'B08', 'B8A', 'B11', 'B12', 'SCL']:
        
        item_band_ref = item.assets[band].get_absolute_href()
        logging.info('Adding {} to vrt'.format(item_band_ref))
        vrt_bands.append(item_band_ref)


    vrt = '{}.vrt'.format(collection.id)
    tif = '{}.tif'.format(collection.id)
    
    logging.info('Build vrt for {}'.format(collection.id))
    
    ds = gdal.BuildVRT(vrt,
                       vrt_bands,
                       srcNodata=0,
                       xRes=10, 
                       yRes=10,
                       separate=True)
    ds.FlushCache()
    
    logging.info('Translate {}'.format(collection.id))

    if bbox:
        
        x_min, y_min, x_max, y_max = bbox
        
        gdal.Translate(tif,
                       vrt,
                       projWin=[x_min, y_max, x_max, y_min],
                       projWinSRS='EPSG:4326',
                       outputType=gdal.GDT_UInt16)
    else:
        
        gdal.Translate(tif,
                       vrt,
                       outputType=gdal.GDT_UInt16)
    os.remove(vrt)


In [ ]:
ds = gdal.Open('pre-event.tif')

pre_b04 = ds.GetRasterBand(1).ReadAsArray()
pre_b08 = ds.GetRasterBand(2).ReadAsArray()
pre_b8a = ds.GetRasterBand(3).ReadAsArray()
pre_b11 = ds.GetRasterBand(4).ReadAsArray()
pre_b12 = ds.GetRasterBand(5).ReadAsArray()
pre_scl = ds.GetRasterBand(6).ReadAsArray()

ds = None

os.remove('pre-event.tif')

    

In [ ]:
ds = gdal.Open('post-event.tif')

post_b04 = ds.GetRasterBand(1).ReadAsArray()
post_b08 = ds.GetRasterBand(2).ReadAsArray()
post_b8a = ds.GetRasterBand(3).ReadAsArray()
post_b11 = ds.GetRasterBand(4).ReadAsArray()
post_b12 = ds.GetRasterBand(5).ReadAsArray()
post_scl = ds.GetRasterBand(6).ReadAsArray()

width = ds.RasterXSize
height = ds.RasterYSize


input_geotransform = ds.GetGeoTransform()
input_georef = ds.GetProjectionRef()

proj = osr.SpatialReference(wkt=ds.GetProjection())
epsg = proj.GetAttrValue('AUTHORITY',1)


ds = None

os.remove('post-event.tif')


In [ ]:
output_files = []

#### RGB Pre and Post event COG (bands: B12,B11,B8A)

In [ ]:
output_name = 'RGB_Pre_{}.tif'.format(pre_event['value'].split('=')[-1])

logging.info('Creating pre-event RGB COG product {}'.format(output_name))

write_RGB([pre_b12,pre_b11,pre_b8a], output_name, width, height, input_geotransform, input_georef)

output_files.append(output_name)

In [ ]:
output_name = 'RGB_Pst_{}.tif'.format(post_event['value'].split('=')[-1])

logging.info('Creating post-event RGB COG product {}'.format(output_name))

write_RGB([post_b12,post_b11,post_b8a], output_name, width, height, input_geotransform, input_georef)

output_files.append(output_name)

In [ ]:
pre_b12 = None
post_b12 = None
pre_b8a = None
post_b8a = None

### NDVI and NDWI Computation

In [ ]:
ndvwi = lambda x,y: 0 if (x+y)==0  else float(x-y)/float(x+y)

vfunc = np.vectorize(ndvwi, otypes=[np.float32])

 #### NDWI with NIR (8) and SWIR (11)

In [ ]:
pre_ndwi2 = vfunc(pre_b08,pre_b11)
post_ndwi2 = vfunc(post_b08,post_b11)

pre_b11 = None
post_b11 = None

#### NDVI with NIR (8) and Red (4)

In [ ]:
pre_ndvi = vfunc(pre_b08,pre_b04)
post_ndvi = vfunc(post_b08,post_b04)

pre_b04 = None
post_b04 = None

pre_b08 = None
post_b08 = None

#### Burned Area computation: 
#### If NDWI i2 - NDWI i1 > 0.18 and If NDVI i2 - NDVI i1 > 0.19 then burned pixels

In [ ]:
ndwi_diff = pre_ndwi2  - post_ndwi2

In [ ]:
ndvi_diff = pre_ndvi - post_ndvi

In [ ]:
conditions = lambda x,y,z,m,n,p: 1 if ((x  > float(y)) & (z > float(m)) & ((n == 4) | (p == 4))) else 0
                             
vfunc_conditions = np.vectorize(conditions, otypes=[np.uint8])

In [ ]:
burned_0 = vfunc_conditions(ndwi_diff, ndwi_threshold['value'], ndvi_diff, ndvi_threshold['value'], pre_scl, post_scl )

In [ ]:
pre_ndwi2 = None
post_ndwi2 = None

pre_ndvi = None
post_ndvi = None

### Exclude according to scene classifications:

where noData put burned=2 if burn then put burned=1 else burned=0

In [ ]:
brnd = lambda x,y,z: 2 if (x==0 or y==0 or x==1 or y==1 or x==6 or y==6 or x==8 or y==8 or x==9 or y==9) else z

vfunc = np.vectorize(brnd, otypes=[np.uint8])

burned = vfunc(pre_scl , post_scl, burned_0 )

In [ ]:
burned_0 = None

##### Write the burned area temp tiff

In [ ]:
#Requested file name : 'Burned_Area_S2_{MasterId}_{SlaveId}.tif

temp_output_name_Burned_Area = 'temp_Burned_Area_S2_%s_%s.tif'%(masterID,slaveID)


In [ ]:
logging.info('Creating Temporary Burned Area COG product {}'.format(temp_output_name_Burned_Area))
write_tif(burned, temp_output_name_Burned_Area, width, height, input_geotransform, input_georef)

##### Post-processing step: removing raster polygons smaller than the provided threshold size (in pixels) - if threshold=0 no post-proc will be applied

In [ ]:
output_name_Burned_Area = '_'.join(temp_output_name_Burned_Area.split('_')[1:])

if int(pp_threshold['value']) != 0:
    logging.info('Creating Sieve-filtered Burned Area COG product {}'.format(output_name_Burned_Area))
    sieve_filter(temp_output_name_Burned_Area,
                 output_name_Burned_Area, 
                 int(pp_threshold['value']))
    
    #os.remove(temp_output_name_Burned_Area)

else:
    
    shutil.move(temp_output_name_Burned_Area,
                output_name_Burned_Area)

logging.info('Burned Area COG product {} created'.format(output_name_Burned_Area))

In [ ]:
output_files.append(output_name_Burned_Area)

##### Creating the mask for the burned area to polygonize only Burned area polygons

In [ ]:
ds = gdal.Open(output_name_Burned_Area)
    
ba = ds.GetRasterBand(1).ReadAsArray()
ds=None

brnd_mask = lambda x: 1 if (x==1) else 0

vfunc = np.vectorize(brnd_mask, otypes=[np.uint8])

mask_burned_area = vfunc(ba)

write_tif(mask_burned_area, 'MASK_burned_area.tif', width, height, input_geotransform, input_georef)


In [ ]:
poligonised_file = '{}_polygonized.json'.format(output_name_Burned_Area.split('.')[0])

logging.info('Creating Burned Area polygonized json file {}'.format(poligonised_file))

change_detection_gp = polygonize(output_name_Burned_Area, poligonised_file, 1, epsg, 'MASK_burned_area.tif')

In [ ]:
output_files.append(poligonised_file)

#### if we replace {'init':'epsg:{}'.format(epsg)} with new recommended 'epsg:{}', the axis order changes

In [ ]:
change_detection_gp.head(10)

### Get the result WKT

In [ ]:
src = gdal.Open(output_name_Burned_Area)
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

max_x = ulx + (src.RasterXSize * xres)
min_y = uly + (src.RasterYSize * yres)
min_x = ulx 
max_y = uly

min_x, min_y, max_x, max_y

In [ ]:
source = osr.SpatialReference()
source.ImportFromWkt(src.GetProjection())

target = osr.SpatialReference()
target.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(source, target)

result_wkt = box(transform.TransformPoint(min_x, min_y)[1],
                 transform.TransformPoint(min_x, min_y)[0],
                 transform.TransformPoint(max_x, max_y)[1],
                 transform.TransformPoint(max_x, max_y)[0]).wkt

In [ ]:
result_wkt

### Create the properties file

In [ ]:
from datetime import datetime

In [ ]:
date_format = '%Y-%m-%dT%H:%m:%SZ'

In [ ]:
output_files

In [ ]:
for index , item in enumerate(output_files):

    if 'RGB' in item:
        prod = slaveID
        if 'Pre' in item[4:7]:
            prod = masterID
        title = 'Sentinel-2 RGB {}-event {} (B11, B12, B8A)'.format(item[4:7],prod)
            
    if 'Burned_Area_S2' in item:
        title = 'Sentinel-2 burned area identification for pair {}/{}'.format(masterID,slaveID)
        if 'temp_' in item:
            title = 'Sentinel-2 burned area identification for pair {}/{} (pre-filtering)'.format(masterID,slaveID)
    
    if 'polygonized' in item:
        title = 'Geojson with vectorization of bitmask burned=1/not burned=0/unkown=2 for pair {}/{}'.format(masterID,slaveID)
        
    
    with open('{}.properties'.format(item), 'w') as file:
        
        file.write('title={}\n'.format(title))
        
        if 'Pre-event' in title:
            date_iso = pre_date
            
            file.write('date={}/{}\n'.format(date_iso,date_iso))
        elif 'Pst-event' in title :
            date_iso = post_date
            
            file.write('date={}/{}\n'.format(date_iso,date_iso))
        else:
            start_date_iso = pre_date
            end_date_iso = post_date
            file.write('date={}/{}\n'.format(start_date_iso,end_date_iso))
            
        file.write('geometry={}'.format(result_wkt))


### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.